In [1]:
# importujemy wymagane biblioteki
import requests

from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver import Firefox

In [2]:
def download_page(page_number: int, job_name: str, browser) -> BeautifulSoup:
    url = 'https://nofluffjobs.com/pl/jobs?criteria={job_name}&page={page_number}'
    browser.get(url.format(
        job_name=job_name,
        page_number=page_number)
        )
    html = browser.page_source

    return BeautifulSoup(html)

In [3]:
def is_empty_page(page) -> bool:
    """
    Sprawdza czy strona jest pusta
    """
    empty_condition = page.find('h2', class_='text-white font-weight-bold')

    if empty_condition is None:
        return False
    else:
        return True

In [4]:
def get_page_jobs(page) -> list:
    """
    Ze strony głównej NoFluffJobs pobiera listę ofer pracy i zwraca list zawieracjącą słownik {job_id, job_url}

    @param page: BeufitifulSoup strony
    @returns: lista ze słownikami {job_id, job_url}
    """

    # upewniamy się, że strona nie jest pusta
    if is_empty_page(page):
        return []

    # znajdujemy oferty pracy dostępne na stronie (przykład znalezienie)
    # poprzez znalezienie obrazka oraz jego rodzica
    jobs = [x.parent for x in page.find_all('div', class_='posting-image')]

    result = []  # zmienna z informacjami wyjściowymi
    for job in jobs:
        id = job['id']  # id oferty
        url = job['href']  # url do oferty

        # dodajemy informacje do zmiennej wynikowej
        result.append({'job_id': id, 'job_url': url})

    return result

In [5]:
def get_job_offers(job_name: str) -> list:
    """
    Pobiera aktualnie dostępne oferty pracy dla danego zapytania

    @param job_name: zapytanie, które chcemy przeszukać
    @returns: lista z ofertami pracy
    """

    results = []  # zmienna z listą ofert dla danego zapytania
    page_number = 1

    # będziemy wykonywać pętle tak długo, aż nie trafimy na pustą stronę
    while True:  
                # pobieramy dane dla danego zawodu oraz strony
        bs = download_page(
            job_name=job_name,
            page_number=page_number)
        
        # sprawdzamy czy strona nie jest pusta
        if is_empty_page(bs):
            break

        # pobieramy informacje o ofertach pracy
        page_jobs = get_page_jobs(bs)
        results += page_jobs

        page_number +=1  # przechodzimy do kolejnej strony
        sleep(5)  # czekamy 5 sekund przed kolejnym odpytaniem

    return results

In [ ]:
jobs = ['data analyst', 'data engineer', 'data scientist', 'python developer']

In [9]:
bs = download_page(
            job_name=jobs[0],
            page_number=1,
            browser=browser)
page_jobs = get_page_jobs(bs)
browser.close()

In [11]:
page_jobs

[{'job_id': 'nfjPostingListItem-D4MZE0LO',
  'job_url': '/pl/job/senior-data-scientist-smartpatient-business-services-warszawa-d4mze0lo'},
 {'job_id': 'nfjPostingListItem-UY4LN9HN',
  'job_url': '/pl/job/data-engineer-mid-senior-smartpatient-business-services-warszawa-uy4ln9hn'},
 {'job_id': 'nfjPostingListItem-MV1QMQOT',
  'job_url': '/pl/job/sas-developer-scs-expert-warszawa-mv1qmqot'},
 {'job_id': 'nfjPostingListItem-YZUNAFAQ',
  'job_url': '/pl/job/data-analyst-iiit-warszawa-yzunafaq'},
 {'job_id': 'nfjPostingListItem-QHPKI9KC',
  'job_url': '/pl/job/senior-data-scientist-upc-polska-remote-qhpki9kc'},
 {'job_id': 'nfjPostingListItem-OE1P5TKN',
  'job_url': '/pl/job/dataops-tech-lead-iiit-remote-oe1p5tkn'},
 {'job_id': 'nfjPostingListItem-VSGLQQR1',
  'job_url': '/pl/job/senior-ppc-specialist-no-fluff-jobs-remote-vsglqqr1'},
 {'job_id': 'nfjPostingListItem-FWV28VKW',
  'job_url': '/pl/job/technical-product-devops-kyp-ai-remote-fwv28vkw'},
 {'job_id': 'nfjPostingListItem-H1BHNV9G',
 